In [ ]:
from scipy import signal
from scipy.stats import linregress

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Excel
from openpyxl import load_workbook
from openpyxl import Workbook
import re
import os 
import glob

from os import listdir

In [ ]:
sns.set_style("white")

### 讀取Excel part

In [ ]:
# Excel
wb = load_workbook(filename=r"C:\Users\BOIL_PO\Desktop\分類部分\VV\基於統計\Time_sheet.xlsx")
ws = wb['Time_VV']

In [ ]:
columns_name = ['No', 'Chart No', 'Date', 'Baseline', 'Stage1', 'Stage2', 'Stage3', 'Stage4', 'Stage5', 'SOFA', 'ECMO']
search_time = pd.DataFrame(columns= columns_name)

for i, row in enumerate(ws['A':'K']):
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    search_time[columns_name[i]] = data_cols


search_time = search_time.dropna().loc[1:]
search_time = search_time.set_index('No')
search_time.head(20)

### 我不知道宇翰要幹嗎???

1. [970] 左腳
2. [973] 右腳

In [ ]:
data_list = glob.glob(r'C:\Users\BOIL_PO\Desktop\分類部分\VV\基於統計\vv_testdata\*.xlsx')
data_list[40]

In [ ]:
def get_df(link):
    df = pd.read_excel(link, skiprows= 65, index_col=0)
    dfs = df.drop(columns=23)
    colname = ['time(ms)', 'L_TSI%', 'L_CH1_HbO2', 'L_CH1_HHb', 'L_CH1_tHb', 'L_CH2_HbO2', 'L_CH2_HHb', 'L_CH2_tHb', 'L_CH3_HbO2', 'L_CH3_HHb', 'L_CH3_tHb', 'R_TSI%', 'R_CH1_HbO2', 'R_CH1_HHb', 'R_CH1_tHb', 'R_CH2_HbO2', 'R_CH2_HHb', 'R_CH2_tHb', 'R_CH3_HbO2', 'R_CH3_HHb', 'R_CH3_tHb']
    dfs.columns = colname
    dfs['time(s)'] = dfs['time(ms)']/1000
    dfs = dfs.set_index('time(s)')
    dff = dfs.drop(columns=['time(ms)'])

    select_side = search_time[search_time['Chart No'] == int(link.split('\\')[-1].split('-')[1])]['ECMO']
    if select_side.values[0] == '左腳':
        print('用右腳data')
        df_r = dff.iloc[:, 10:]
        df_r.columns = ['TSI%', 'CH1_HbO2', 'CH1_HHb', 'CH1_tHb', 'CH2_HbO2', 'CH2_HHb', 'CH2_tHb', 'CH3_HbO2', 'CH3_HHb', 'CH3_tHb']
        return df_r
        
    else:
        print('用左腳data')
        df_r = dff.iloc[:, :10]
        df_r.columns = ['TSI%', 'CH1_HbO2', 'CH1_HHb', 'CH1_tHb', 'CH2_HbO2', 'CH2_HHb', 'CH2_tHb', 'CH3_HbO2', 'CH3_HHb', 'CH3_tHb']
        return df_r


In [ ]:
dfm = get_df(data_list[21])
dfm

In [ ]:
plt.figure(figsize= (10, 10))
for i, cols in enumerate(dfm.columns):
    plt.subplot(5, 2, i+1)
    plt.plot(dfm.index, dfm[cols], label= f'{cols}')
    plt.title(cols)
    plt.tight_layout()
    plt.legend()

In [ ]:
def filters(df_col):
    b, c = signal.butter(4, 0.01, 'lowpass')
    filter_a = signal.filtfilt(b, c,  df_col)
    return filter_a

In [ ]:
dfyy = dfm.apply(filters)
plt.figure(figsize= (10, 10))
for i, cols in enumerate(dfyy.columns):
    plt.subplot(5, 2, i+1)
    plt.plot(dfyy.index, dfyy[cols], label= f'{cols}')
    plt.title(cols)
    plt.tight_layout()
    plt.legend()

In [ ]:
from TDDR import TDDR

In [ ]:
def tddr(x):
    return TDDR(x, 17)

In [ ]:
dftt = dfyy.apply(tddr)
plt.figure(figsize= (10, 10))
for i, cols in enumerate(dftt.columns):
    plt.subplot(5, 2, i+1)
    plt.plot(dftt.index, dftt[cols], label= f'{cols}')
    plt.title(cols)
    plt.tight_layout()
    plt.legend()

In [ ]:
def minmax(df):
    df_out = (df - df.min())/(df.max()- df.min())

    return df_out

In [ ]:
dfzz = dftt.apply(minmax)
plt.figure(figsize= (10, 10))
for i, cols in enumerate(dfzz.columns):
    plt.subplot(5, 2, i+1)
    plt.plot(dfzz.index, dfzz[cols], label= f'{cols}')
    plt.title(cols)
    plt.tight_layout()
    plt.legend()

## 批次行為

In [ ]:
for data_link in data_list:

    dfm = get_df(data_link)
    dfyy = dfm.apply(filters)
    dftt = dfyy.apply(tddr)
    dfzz = dftt.apply(minmax)

    plt.figure(figsize= (10, 10))
    for i, cols in enumerate(dfzz.columns):
        plt.subplot(5, 2, i+1)
        plt.plot(dfzz.index, dfzz[cols], label= f'{cols}')
        plt.title(cols)
        plt.tight_layout()
        plt.legend()
    plt.show()

    dfzz.to_csv(data_link.replace('.xlsx', '_processed.csv').replace('vv_testdata', 'vv_clean'), index= True)    